In [ ]:
# import libraries

import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model

In [ ]:
# run the data_processing file to get the training and validation sets

%run ../data_processing.ipynb

In [ ]:
# compile parameters (can be updated for different variations and test)
X_TRAIN = x_train
Y_TRAIN = y_train

X_VALID = x_valid
Y_VALID = y_valid

X_TEST = x_test
Y_TEST = y_test

VALIDATION = (x_valid, y_valid)

ACTIVATION_FUNCTION = 'relu'
ACTIVATION_FUNCTION_OUTPUT = 'softmax'
BATCH_SIZE = 32
EPOCHS = 50
INPUT_DIM = 12
KERNEL_INITIALIZER = 'he_uniform'
LEARNING_RATE = 0.03
LOSS_FUNCTION = 'categorical_crossentropy'
METRICS = ['accuracy']
VERBOSE = 0


# list of optimizers to test (can be updated to test different optimizers)
optimizers = {'SGD': keras.optimizers.SGD(learning_rate=LEARNING_RATE),
              'RMSprop': keras.optimizers.RMSprop(learning_rate=LEARNING_RATE),
              'Adam': keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              'AdamW': keras.optimizers.AdamW(learning_rate=LEARNING_RATE),
              'Adadelta': keras.optimizers.Adadelta(learning_rate=LEARNING_RATE),
              'Adagrad': keras.optimizers.Adagrad(learning_rate=LEARNING_RATE),
              'Adamax': keras.optimizers.Adamax(learning_rate=LEARNING_RATE),
              'Adafactor': keras.optimizers.Adafactor(learning_rate=LEARNING_RATE),
              'Nadam': keras.optimizers.Nadam(learning_rate=LEARNING_RATE),
              'Ftrl': keras.optimizers.Ftrl(learning_rate=LEARNING_RATE),
              }

# dataframe to hold the loss and accuracy for each optimizer
df_result = pd.DataFrame(columns=['optimizer', 'training_loss',
                                  'training_accuracy', 'validation_loss',
                                  'validation_accuracy', 'test_loss',
                                  'test_accuracy'])

# clears the dataset to avoid redundancy
df_result = df_result.iloc[0:0]

In [ ]:
# create a baseline classifier
def create_baseline():

    # init a sequential NN
    classifier = Sequential()

    # Define the model architecture
    
    # 1st layer - takes in input
    classifier.add(Dense(units=512, kernel_initializer=KERNEL_INITIALIZER,
                         activation=ACTIVATION_FUNCTION, input_dim=INPUT_DIM))
    # 2nd layer
    classifier.add(Dense(units=128, kernel_initializer=KERNEL_INITIALIZER,
                         activation=ACTIVATION_FUNCTION))
    # 3rd layer
    classifier.add(Dense(units=96, kernel_initializer=KERNEL_INITIALIZER,
                         activation=ACTIVATION_FUNCTION))
    # output layer
    classifier.add(Dense(units=2, kernel_initializer=KERNEL_INITIALIZER,
                         activation=ACTIVATION_FUNCTION_OUTPUT))

    return classifier

In [ ]:
# loop through the optimizers list to test all the optimizers
for OPTIMIZER in optimizers:
    
    # create a baseline model
    model = create_baseline()

    # compile the model with values defined above
    model.compile(optimizer=optimizers[OPTIMIZER],
                  loss=LOSS_FUNCTION, metrics=METRICS)
    
    # train the model
    history = model.fit(X_TRAIN, Y_TRAIN, validation_data=VALIDATION,
                            epochs=EPOCHS, batch_size=BATCH_SIZE,
                            verbose=VERBOSE)

    # plot
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid = True
    plt.ylim(0, 1)
    plt.title('Optimizer: ' + str(OPTIMIZER))
    plt.savefig('figures/optimizer_'+ OPTIMIZER +'.png')
    plt.show()

    # evaluation - loss and accuracy
    model_train_loss, model_train_accuracy = model.evaluate(X_TRAIN, Y_TRAIN,
                                                            verbose=VERBOSE)
    model_valid_loss, model_valid_accuracy = model.evaluate(X_VALID, Y_VALID,
                                                            verbose=VERBOSE)
    model_test_loss, model_test_accuracy = model.evaluate(X_TEST, Y_TEST,
                                                          verbose=VERBOSE)

    # add the evaluation results to the dataset
    df_result = pd.concat([pd.DataFrame([[OPTIMIZER,
                                          model_train_loss,
                                          model_train_accuracy,
                                          model_valid_loss,
                                          model_valid_accuracy,
                                          model_test_loss,
                                          model_test_accuracy]],
                                          columns=df_result.columns),
                                          df_result],
                                          ignore_index=True)

In [ ]:
# display the evaluation results dataframe

df_result

In [ ]:
# visualize loss for training, validation and test sets

# x values
x = ['training_loss', 'validation_loss', 'test_loss']

# training and validation loss for each optimizer
for _idx in range(df_result.shape[0]):
    plt.plot(x, [df_result.iloc[_idx]['training_loss'],
                 df_result.iloc[_idx]['validation_loss'],
                 df_result.iloc[_idx]['test_loss']],
                 label=str(df_result.iloc[_idx]['optimizer']))

# plot
plt.grid = True
plt.ylim(0, 1)
plt.title('Model Loss with varying Optimizers')
plt.legend(loc='upper right')
plt.savefig('figures/loss.png')
plt.show()

In [ ]:
# visualize accuracy for training, validation and test sets

# x values
x = ['training_accuracy', 'validation_accuracy', 'test_accuracy']

# training and validation accuracy for each optimizer
for _idx in range(df_result.shape[0]):
    plt.plot(x, [df_result.iloc[_idx]['training_accuracy'],
                 df_result.iloc[_idx]['validation_accuracy'],
                 df_result.iloc[_idx]['test_accuracy']],
                 label=str(df_result.iloc[_idx]['optimizer']))

# plot
plt.grid = True
plt.ylim(0, 1)
plt.title('Model Accuracy with varying Optimizers')
plt.legend(loc='upper right')
plt.savefig('figures/accuracy.png')
plt.show()